# Introdução ao Django: Python 

[Aprenda Python com Jupyter](https://github.com/jeanto/python_django_course_notebook) by [Jean Nunes](https://jeanto.github.io/jeannunes)   
Code license: [GNU-GPL v3](https://www.gnu.org/licenses/gpl-3.0.en.html)

---

# Validações

**1. Classe `Validador` e subclasses:**

- A classe `Validador` e suas subclasses implementam a lógica de validação para os campos, de acordo com os requisitos de negócio.
    
- Cada validador verifica se o valor do campo atende aos critérios especificados e levanta uma exceção `ValidationError` se a validação falhar.

- O método `validar` em cada classe da cadeia chama o próximo validador na cadeia, se houver um. Isso permite que vários validadores sejam encadeados para realizar múltiplas validações em um único campo.


**Arquivo `validador.py`:**

In [ ]:
from django.core.exceptions import ValidationError
import re

class Validador:
    """
    Classe base para os validadores.
    """
    def __init__(self, proximo_validador=None):
        self.proximo_validador = proximo_validador

    def validar(self, value, field_name):
        """
        Método para validar o valor de um campo.
        Deve ser implementado nas subclasses.

        Args:
            value: O valor a ser validado.
            field_name (str): O nome do campo que está sendo validado.

        Returns:
            None: Se a validação for bem-sucedida.

        Raises:
            ValidationError: Se a validação falhar.
        """
        if self.proximo_validador:
            self.proximo_validador.validar(value, field_name)

class ValidadorNome(Validador):
    def validar(self, value, field_name):
        super().validar(value, field_name)
        if not isinstance(value, str) or not value.strip() or not value.replace(" ", "").isalpha():
            raise ValidationError(f"{field_name} deve ser uma string não vazia e conter apenas letras.")

class ValidadorScriptInjection(Validador):
    def validar(self, value, field_name):
        super().validar(value, field_name)
        if re.search(r'<script.*?>.*?</script>', value, re.IGNORECASE):
            raise ValidationError(f"{field_name} contém tags de script e não é permitido.")

class ValidadorXSS(Validador):
    def validar(self, value, field_name):
        super().validar(value, field_name)
        if re.search(r'<[^>]+>', value):
            raise ValidationError(f"{field_name} contém tags HTML e não é permitido.")
        
### https://regex101.com/r/qE9gR7/1
class ValidadorSQLInjection(Validador):
    def validar(self, value, field_name):
        super().validar(value, field_name)

        regex = r"(\s*([\0\b\'\"\n\r\t\%\_\\]*\s*(((select\s*.+\s*from\s*.+)|(insert\s*.+\s*into\s*.+)|(update\s*.+\s*set\s*.+)|(delete\s*.+\s*from\s*.+)|(drop\s*.+)|(truncate\s*.+)|(alter\s*.+)|(exec\s*.+)|(\s*(all|any|not|and|between|in|like|or|some|contains|containsall|containskey)\s*.+[\=\>\<=\!\~]+.+)|(let\s+.+[\=]\s*.*)|(begin\s*.*\s*end)|(\s*[\/\*]+\s*.*\s*[\*\/]+)|(\s*(\-\-)\s*.*\s+)|(\s*(contains|containsall|containskey)\s+.*)))(\s*[\;]\s*)*)+)"

        if re.search(regex, value, re.IGNORECASE):
            raise ValidationError(f"{field_name} contém caracteres que podem ser usados para injeção SQL.")
        

**Arquivo `models.py`:**

In [ ]:
from django.db import models
from django.core.validators import MinValueValidator, MaxValueValidator, ValidationError
from .validador import ValidadorNome, ValidadorScriptInjection, ValidadorXSS, ValidadorSQLInjection # Importe os validadores do arquivo validators.py
import re

def validar_cpf(value):
    """Função para validar o CPF."""
    # Remove caracteres não numéricos
    cpf = ''.join(filter(str.isdigit, value))

    # Lógica de validação do CPF (adaptada de https://www.pythonbrasil.org.br/wiki/VerificadorDeCPF)
    try:
        int(cpf)  # Verifica se todos os caracteres são números
    except ValueError:
        raise ValidationError("CPF deve conter apenas números.")

    # Cálculo do primeiro dígito verificador
    soma = 0
    for i in range(9):
        soma += int(cpf[i]) * (10 - i)
    resto = soma % 11
    digito1 = 0 if resto < 2 else 11 - resto

    if digito1 != int(cpf[9]):
        raise ValidationError("CPF inválido.")

    # Cálculo do segundo dígito verificador
    soma = 0
    for i in range(10):
        soma += int(cpf[i]) * (11 - i)
    resto = soma % 11
    digito2 = 0 if resto < 2 else 11 - resto

    if digito2 != int(cpf[10]):
        raise ValidationError("CPF inválido.")
    
    # Se o CPF for válido, retorna o valor original
    return cpf

class Pessoa(models.Model):
    id = models.AutoField(primary_key=True)
    nome = models.CharField(max_length=255)
    idade = models.IntegerField(
        validators=[
            MinValueValidator(0, message="A idade deve ser maior ou igual a 0."),
            MaxValueValidator(125, message="A idade deve ser menor ou igual a 125."),
        ]
    )
    sexo = models.CharField(max_length=1, choices=[('M', 'Masculino'), ('F', 'Feminino')])
    data_nascimento = models.DateField()
    cidade_natal = models.CharField(max_length=100)
    estado_natal = models.CharField(max_length=2)
    cpf = models.CharField(max_length=14, unique=True, validators=[validar_cpf])
    profissao = models.CharField(max_length=100, blank=True, null=True)
    cidade_residencia = models.CharField(max_length=100)
    estado_residencia = models.CharField(max_length=2)
    estado_civil = models.CharField(max_length=50, choices=[
         ('Solteiro', 'Solteiro'), ('Casado', 'Casado'), 
         ('Divorciado', 'Divorciado'), ('Viuvo', 'Viuvo'), 
         ('Uniao Estavel', 'Uniao Estavel')])


    # O método clean() no Django é usado para validação personalizada de modelos. 
    # Ele permite que você defina regras de validação além das validações 
    # padrão dos campos. O Django chama esse método quando você usa full_clean() 
    # em uma instância do modelo, geralmente antes de salvar no banco de dados.
    def clean(self):
        super().clean()
        validador_nome = ValidadorNome()

        # Validar o nome
        validador_nome.validar(self.nome, 'Nome')

    class Meta:
        abstract = True  # Classe base abstrata, não será criada tabela no banco

    def __str__(self):
        return f"{self.nome} ({self.cpf})"

    @classmethod
    def cadastrar(cls, **kwargs):
        """
        Método de classe para cadastrar uma pessoa.
        Deve ser implementado nas subclasses.
        """
        raise NotImplementedError("O método cadastrar deve ser implementado na subclasse.")
    
    def editar(cls, **kwargs):
        """
        Método de classe para editar uma pessoa.
        Deve ser implementado nas subclasses.
        """
        raise NotImplementedError("O método editar deve ser implementado na subclasse.")

class Doador(Pessoa):
    contato_emergencia = models.CharField(max_length=255)
    tipo_sanguineo = models.CharField(max_length=5, choices=[
        ('A+', 'A+'), ('A-', 'A-'), ('B+', 'B+'), ('B-', 'B-'),
        ('AB+', 'AB+'), ('AB-', 'AB-'), ('O+', 'O+'), ('O-', 'O-')
    ])

    def clean(self):
        super().clean()
        # Validaçoes de segurança
        validador_xss       = ValidadorXSS()
        validador_script    = ValidadorScriptInjection(validador_xss)
        validador_sql       = ValidadorSQLInjection(validador_script)

        # Valida todos os campos
        for field in self._meta.fields:
            if field.name != 'id':
                value = getattr(self, field.name)
                if value:
                    # Valida o valor do campo usando o validador apropriado
                    validador_sql.validar(str(value), field.name)

    @classmethod
    def cadastrar(cls, dados):
        """
        Método de classe para cadastrar um doador.
        """
        try:
            doador = cls(**dados)

            doador.full_clean()  # Valida o model
            doador, criado = Doador.objects.update_or_create(
                cpf=dados['cpf'], # Campo chave para identificar o doador
                defaults={k: v for k, v in dados.items() if k != 'cpf'}
            )
            return doador, criado, None  # Retorna o objeto, se foi criado e nenhum erro
        except ValidationError as e:
            return None, False, e.message_dict # Retorna None, False e os erros de validação
    
    def editar(doador, **kwargs):
        """
        Método de classe para editar um doador.
        """
        try:
            doador.full_clean()  # Valida o model
            doador.save()  # Salva as alterações
            return doador, True, None  # Retorna o objeto e nenhum erro
        except ValidationError as e:
            return None, False, e.message_dict


**2. Validação no método `clean` do Model:**

- O método `clean` é um método do Django Model que permite realizar validações personalizadas em um Model.
    
- Nos Models `Pessoa` e `Doador`, o método `clean` foi sobrescrito para executar a cadeia de validadores.
    
- A cadeia de validadores é criada instanciando os validadores na ordem em que devem ser executados e passando o próximo validador na cadeia para o construtor do validador anterior.
    
- O método `validar` do primeiro validador na cadeia é chamado, passando o valor do campo a ser validado. Se a validação falhar em algum ponto da cadeia, uma exceção `ValidationError` será levantada.

**3. Validação do CPF:**

- A função `validar_cpf` foi mantida para realizar a validação específica do CPF.
    
- Essa função é usada como um validador no campo cpf do Model Pessoa.

**4. Chamada de `full_clean()` no método cadastrar:**    
    
- No método `cadastrar` do Model `Doador`, o método `full_clean()` é chamado antes de salvar o objeto no banco de dados. Isso garante que todas as validações definidas no Model, incluindo as validações da cadeia de validadores e a validação do CPF, sejam executadas.

O código agora implementa o padrão *Chain of Responsibility* para validar o campo `nome` e tratar questões de segurança para todos os campos no Django Model `Pessoa` e `Doador`. A validação do CPF continua sendo feita pela função `validar_cpf`, que é usada como um validador no campo cpf do Model.